# 04. Ontology-Inspired Evaluation

Combine ML anomaly scores with domain-knowledge-based ontology penalties and compare performance.

In [ ]:
# === UNIVERSAL PATH SETUP (Works in both Local and Colab) ===
import sys
import os

# Auto-detect environment and setup paths
try:
    from src.utils import setup_paths
    env_type = setup_paths()
except ImportError:
    # Fallback if utils not found (first run)
    print("⚙️  Setting up paths...")
    try:
        import google.colab
        in_colab = True
        if 'notebooks' in os.getcwd():
            os.chdir('..')
        project_root = os.getcwd()
        print("☁️  Detected: Google Colab")
    except ImportError:
        in_colab = False
        project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        print("💻 Detected: Local Environment")
    
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    print(f"✅ Project root: {project_root}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from src.preprocessing import build_feature_matrix, train_test_split_stratified, get_selected_features, clean_data, create_target
from src.models import fit_isolation_forest, score_isolation_forest
from src.ontology import apply_ontology_rules, combine_scores
from src.evaluation import evaluate_anomaly_detector, plot_roc_pr, save_metrics_summary, print_comparison_table

# Create results directory
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True)

## Step 1: Load Raw Data (with original features for ontology rules)

In [ ]:
# We need the original data to apply ontology rules
from src.preprocessing import load_raw_data

data_path = '../data/raw/diabetic_data.csv'
df_raw = load_raw_data(data_path)

# Clean and prepare
selected_features = get_selected_features()
df_clean = clean_data(df_raw, selected_features)
X_features, y = create_target(df_clean)

print(f"Data shape: {X_features.shape}")
print(f"Target distribution: {y.value_counts()}")

## Step 2: Build Feature Matrix and Split Data

In [ ]:
# Build preprocessed features
X, y, preprocessor = build_feature_matrix(data_path)

# Split (use same indices for both raw and processed)
X_train, X_test, y_train, y_test = train_test_split_stratified(X, y, test_size=0.2, random_state=42)

# Get corresponding raw features for test set
test_indices = X_test.index
X_features_test = X_features.loc[test_indices]

print(f"Test set size: {X_test.shape[0]}")

## Step 3: Train Isolation Forest and Get Scores

In [ ]:
# Train model
print("Training Isolation Forest...")
iso_forest = fit_isolation_forest(X_train.values, contamination=0.1, random_state=42)

# Get anomaly scores
if_scores_test = score_isolation_forest(iso_forest, X_test.values)
print("✓ Model trained and scores computed!")

## Step 4: Compute Ontology Penalties

In [ ]:
# Apply ontology rules to raw test features
print("Applying ontology rules...")
ontology_penalties = apply_ontology_rules(X_features_test)

print(f"\nOntology penalty distribution:")
print(ontology_penalties.value_counts().sort_index())
print(f"\nMean penalty: {ontology_penalties.mean():.4f}")

## Step 5: Combine ML Scores with Ontology Penalties

In [ ]:
# Combine scores (alpha=0.7 for ML, beta=0.3 for ontology)
combined_scores = combine_scores(if_scores_test, ontology_penalties.values, alpha=0.7, beta=0.3)

print(f"Combined scores range: [{combined_scores.min():.4f}, {combined_scores.max():.4f}]")
print(f"Mean combined score: {combined_scores.mean():.4f}")

## Step 6: Evaluate Both Approaches

In [ ]:
# Evaluate ML-only
metrics_ml_only = evaluate_anomaly_detector(y_test.values, if_scores_test, model_name="Isolation Forest (ML Only)")

# Evaluate combined
metrics_combined = evaluate_anomaly_detector(y_test.values, combined_scores, model_name="IF + Ontology")

## Step 7: Comparison

In [ ]:
# Print comparison table
print_comparison_table([metrics_ml_only, metrics_combined])

# Calculate improvement
roc_improvement = ((metrics_combined['roc_auc'] - metrics_ml_only['roc_auc']) / metrics_ml_only['roc_auc']) * 100
pr_improvement = ((metrics_combined['pr_auc'] - metrics_ml_only['pr_auc']) / metrics_ml_only['pr_auc']) * 100

print(f"\n{'='*60}")
print(f"PERFORMANCE IMPROVEMENT")
print(f"{'='*60}")
print(f"ROC-AUC improvement: {roc_improvement:+.2f}%")
print(f"PR-AUC improvement:  {pr_improvement:+.2f}%")
print(f"{'='*60}\n")

In [ ]:
# Plot comparison
plot_roc_pr(y_test.values, {
    'IF (ML Only)': if_scores_test,
    'IF + Ontology': combined_scores
})
plt.savefig(results_dir / 'ontology_comparison_curves.png', dpi=150, bbox_inches='tight')
print(f"✓ Comparison plots saved to {results_dir / 'ontology_comparison_curves.png'}")

## Step 8: Save Final Results

In [ ]:
# Save comparison metrics
save_metrics_summary([metrics_ml_only, metrics_combined], results_dir / 'ontology_comparison_metrics.csv')
print("✓ Ontology evaluation complete!")

## Summary

This notebook demonstrated:
1. **Ontology-Inspired Rules**: Applied 3 clinical rules to compute domain-knowledge penalties
2. **Score Combination**: Combined normalized ML scores with ontology penalties (α=0.7, β=0.3)
3. **Performance Comparison**: Evaluated "ML Only" vs "ML + Ontology" approaches

**Key Takeaway**: The ontology layer adds clinical domain knowledge to ML predictions, potentially improving identification of high-risk readmission cases.